<a href="https://colab.research.google.com/github/dlz256/ENSE480-Project/blob/main/ENSE480Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Author: Dillan Zurowski
# SID: 200431334
# Class: ENSE 480 Knowledge Base and Information Systems


# connect to google sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
#import libraries
import pandas as pd #pandas for dataframes
import numpy as np  #numpy for filtering out unique words
import re           #re for getting only alphabetic letters

#initialize variables
# wb: workbook (google sheets)
# w: set of all values in the sheet
# df: dataframe used to read data, mostly for finding null values
wb = gc.open("Word")
w = wb.sheet1.get_all_values()
df = pd.DataFrame.from_records(w)
# Here, get a set of unique words in variable v
nested_lst_of_tuples = [tuple(l) for l in w]
nested_lst_of_tuples.sort(key=lambda tup: tup[1], reverse=True)
tup = set(nested_lst_of_tuples)
words = [x[0] for x in tup]
v = set(words)

# The program should be able to update during runtime so resetting these values should be able to be
# accessed at any time
def open_sheet():
  wb = gc.open("Word")
  w = wb.sheet1.get_all_values()
  nested_lst_of_tuples = [tuple(l) for l in w]
  nested_lst_of_tuples.sort(key=lambda tup: tup[1], reverse=True)
  tup = set(nested_lst_of_tuples)
  words = [x[0] for x in tup]
  v = set(words)

# ws: worksheet used for updating cells
ws = gc.open('Word').sheet1

# When a suggested word is used, increase the probability of it showing up first
# first, find the index of the suggested word, then update the cell in the worksheet
def increase_prob(value):
  index= int(get_index(value))-1
  cells = ws.range('B1:B4000')
  cells[index].value = int(cells[index].value) +1
  ws.update_cells(cells)
  open_sheet()
# increase_prob("ability")

# Function that retrievs the probability of a word
def get_prob(word):
   for i in range(len(w)):
     if w[i][0] == word:
       return w[i][1]
   return 1

# Function that gets the index of a word
def get_index(word):
    for i in range(len(w)):
     if w[i][0] == word:
       return w[i][2]

# getProb = get_prob('abandon')
# print("TEST: ",getProb)

# The following four functions are methods of Natural Language Processing(NLP)
# which is a branch of AI that attempts to replicate the human language

# 1. DeleteLetter:removes a letter from a given word
def DeleteLetter(word):
    delete_list = []
    split_list = []
    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))
    for a, b in split_list:
        delete_list.append(a + b[1:])
    return delete_list

# 2. SwitchLetter:swap two adjacent letters
def SwitchLetter(word):
    split_l = []
    switch_l = []
    for i in range(len(word)):
        split_l.append((word[0:i], word[i:]))
        for a,b in split_l:
          if len(b)>=2:
            switch_l.append(a + b[1] + b[0] + b[2:])
    return switch_l
# switch_word_l = SwitchLetter(word="eta")

# 3. replace_letter: changes each letter in the word to every letter in the alphabet
def replace_letter(word):
    split_l = []
    replace_list = []
    for i in range(len(word)):
        split_l.append((word[0:i], word[i:]))
    alphabets = 'abcdefghijklmnopqrstuvwxyz'
    for l in alphabets:
      for a,b in split_l:
        replace_list.append(a + l + (b[1:] if len(b) > 1 else ''))
    return replace_list
# replace_l = replace_letter(word='can')

# 4. insert_letter: adds additional characters before and after every lette
def insert_letter(word):
    split_l = []
    insert_list = []
    for i in range(len(word) + 1):
        split_l.append((word[0:i], word[i:]))
    letters = 'abcdefghijklmnopqrstuvwxyz'
    for l in letters:
      for a, b in split_l:
       insert_list.append(a+l+b)
    return insert_list
# insert_letter = insert_letter("bahh")

# switch operation optional
def edit_one_letter(word):
    edit_set1 = set()
    edit_set1.update(DeleteLetter(word))
    edit_set1.update(SwitchLetter(word))
    edit_set1.update(replace_letter(word))
    edit_set1.update(insert_letter(word))
    # print(edit_set1)
    return edit_set1

# edit the edit
def double_edit(word):
    edit_set2 = set()
    edit_one = edit_one_letter(word)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w)
            edit_set2.update(edit_two)
    # print(edit_set2)
    return edit_set2

# Get a list of reccomended suggestions
# get the list of edits, most are not words so get the list of real words and
# sort them by probability
def get_corrections(word, vocab):
    suggested_word = []
    best_suggestion = []
    suggested_word = list(edit_one_letter(word).intersection(vocab)
                          or double_edit(word).intersection(vocab))
    # print(suggested_word)
    best_suggestion = [s for s in list(reversed(suggested_word))]
    sorted = []
    for x in best_suggestion:
      sorted.append((x, get_prob(x)))
    sorted.sort(key=lambda tup: tup[1], reverse=True)  
    return sorted

# Get the list of surrounding words accociated with a word
def get_surroounding_words(word):
  row = int(get_index(word))
  # print(row)
  wb = gc.open("Word")
  rows = wb.sheet1.get_all_values()
  df = pd.DataFrame.from_records(rows)
  surr_words = []
  for i in range(3, len(df.loc[row-1])):
    surr_words.append(df.loc[row-1][i])
  return(surr_words)

# get_surroounding_words("a")

# This function is to add to the list of words that may appear around a word. 
# This adds more heristics to the data which makes the program more accurate
def add_surrounding_word(baseWord, surroundingWord):
  row= int(get_index(baseWord))
  cell_range = "D{}:Z{}".format(row, row)
  cells = ws.range(cell_range) 

  wb = gc.open("Word")
  rows = wb.sheet1.get_all_values()
  df = pd.DataFrame.from_records(rows)
  surrounding_words = get_surroounding_words(baseWord)
  if surroundingWord not in surrounding_words:
    size =0
    for index in range(len(df.loc[row-1])):
      if df.loc[row-1][index] !='':
        size +=1
    cells[size-3].value = surroundingWord 
    ws.update_cells(cells)
    # print("adding surrounding", surroundingWord, "to ", rows[row-1])
    open_sheet()
  # else: print("Already in")
  # print(w)

# Add a words to the list of words. Input can be any number of words.
# Here np is used to get unique values as to not add a word more then once if it 
# occurs in the list more then once
def add_words(sentance_array):
  count = 1
  sentance_array = np.array(sentance_array)
  sentance_array = np.unique(sentance_array)
  for i in range(len(sentance_array)):
    if sentance_array[i] not in v:
      index = int(len(w)) + count
      count+=1
      cells = ws.range('A{}:C{}'.format(index, index))
      cells[0].value = sentance_array[i]
      cells[1].value = 1
      cells[2].value = index
      ws.update_cells(cells)
      open_sheet()
      
# sentance = ["zzz", "z2", "z3"]
# add_words(sentance)
# add_surrounding_word("able", "test2")

# Option 2 of the autocorrect feature is to use the surrounding words as a parameter
# to determine the probability of new word being the correct replacement.
# This workd by generating a list of temporary correctoins using the 4 functions 
# above, and gets the list of words that have surrounded the words in the past. 
# if the list of surrounding words matches the word that surrounds the current 
# incorrect word, then the probability will increase by 5.
def correct_using_neighbors ():
  for index in range(len(my_array)-1):
    word = my_array[index]
    tmp_corrections = get_corrections(word, v)  #tuple[(word, prob), ...]
    if tmp_corrections:
      sorted_suggestions = []
      for i in range(len(tmp_corrections)):
        surr_words = get_surroounding_words(tmp_corrections[i][0])
        if tmp_corrections[i][1]:
          score = int(tmp_corrections[i][1])
        if index !=0 and my_array[index-1] in surr_words:
          score +=5
        if index != len(my_array) and my_array[index+1] in surr_words:
          score +=5
        sorted_suggestions.append([tmp_corrections[i][0], score])
      sorted_suggestions.sort(key=lambda tup: tup[1], reverse=True)
      my_array[index] = sorted_suggestions[0][0]
  return my_array
 
# This is where the main program will run. 
# Get a user input of a string, select the desired method of autocorrect, either
# using the neighbors and correcting all words automatically, or by giving the
# the list of suggested words and allowing them to select one of them. If they 
# choose the second option, then the probability of word they select will increase
x=True
while(x):
    my_sentance = input("Enter Text:")
    my_sentance = my_sentance.lower()
    my_array = my_sentance.split()
    auto_correct = input("Correct all? (y/n)")
    if auto_correct.lower() == "y":
      my_array = correct_using_neighbors()
    else:
      for index, word in enumerate(my_array):
        if word not in words:
          tmp_corrections = get_corrections(word, v)
          print("Select a Replacement for the word: " + word)
          print("0: Add to dictionary")
          for i, word_prob in enumerate(tmp_corrections):
            print(str(i+1)+": " + word_prob[0])
            if i==3:
              break
          selection= input()
          if int(selection) == 1:
            print("changing "+ my_array[index] + " to " + tmp_corrections[0][0])
            my_array[index] = tmp_corrections[0][0]
            old_prob = tmp_corrections[0][1]
            new_prob = int(old_prob) +1
            # print(f"Changing probability from {old_prob} to {new_prob}")
            increase_prob(tmp_corrections[0][0])
          elif int(selection) == 2:
            print("changing "+ my_array[index] + " to " + tmp_corrections[1][0])
            my_array[index] = tmp_corrections[1][0]
            old_prob = get_prob(tmp_corrections[1][1])
            new_prob = int(old_prob) +1
            # print(f"Changing probability from {old_prob} to {new_prob}")
            increase_prob(tmp_corrections[1][0])
          elif int(selection) == 3:
            print("changing "+ my_array[index] + " to " + tmp_corrections[2][0])
            my_array[index] = tmp_corrections[2][0]
            old_prob = get_prob(tmp_corrections[2][1])
            new_prob = int(old_prob) +1
            # print(f"Changing probability from {old_prob} to {new_prob}")
            increase_prob(tmp_corrections[2][0])
          elif int(selection) == 4:
            print("changing "+ my_array[index] + " to " + tmp_corrections[3][0])
            my_array[index] = tmp_corrections[3][0]
            old_prob = get_prob(tmp_corrections[3][1])
            new_prob = int(old_prob) +1
            # print(f"Changing probability from {old_prob} to {new_prob}")
            increase_prob(tmp_corrections[3][0])
          elif int(selection) == 0:
            add_words(word)
            print("Adding", word, "to dictionary")
          else:
            print('Invalid input!')
            break

    corrected_sentance = ' '.join(my_array)
    print("Corrected:",corrected_sentance)
      

############################
# Optional to increase surrounding words and add new words. Enter in correct sentances. 
def add_to_training(sentance):
  regex = re.compile('[^a-zA-Z]')
  sentance = input("Enter training set")
  sentance = regex.sub(' ', sentance)
  sentance = sentance.lower()
  sentance = sentance.split()
  # print(len(sentance))
  add_words(sentance)
  for index in range(len(sentance)):
    # print("INDEX:",index)
    # print("WORD",sentance[index])
    if index >=1:
      try: add_surrounding_word(sentance[index], sentance[index-1])
      except: continue
      # print("Adding surrounding word!", sentance[index])
    if index < len(sentance)-1:
      try: add_surrounding_word(sentance[index], sentance[index+1])
      except: continue
    if index == len(sentance)-1: break
      # print("Adding surrounding Word:", sentance[index])


SyntaxError: ignored